## Aufgaben

- [x] Adresse -> Coord
- [x] (Adresse, Adresse) -> Route
- [x] Route -> Karte

In [1]:
## client library
import requests
from requests.auth import HTTPBasicAuth

from pprint import pprint


def run_request(url, query, credentials):
    headers = {'Content-type': 'application/json', 'Accept': 'application/json'}
    r = requests.post(url,
                      headers=headers,
                      json=query,
                      auth=HTTPBasicAuth(credentials['user'], credentials['pw']))
    return r.json()


def order_coords(coords):
    return [coords['y'], coords['x']]

def order_polyline(polyline):
    return [order_coords(coords) for coords in polyline]

def get_geocoord(q, credentials, xServerUrl):
    query = {
      "$type": "SearchByTextRequest",
      "text": q
      }
    prepared = run_request(xServerUrl + "/services/rs/XLocate/searchLocations",
                           query,
                           credentials)['results']
    if prepared:
        return prepared[0]['location']['referenceCoordinate']
    

def calc_route(start, finish, xserverUrl):
    def waypoint(x):
        return {
                "$type": "OffRoadWaypoint",
                "location": {
                "offRoadCoordinate": x}
               }

    query = {"waypoints": [waypoint(start), waypoint(finish)],
             "resultFields": {"polyline": True}
            }
    prepared = run_request(xServerUrl + "/services/rs/XRoute/calculateRoute", query, credentials)
    if prepared:
        return prepared['polyline']['plain']['polyline']

In [2]:
# task 1
credentials = dict(user = 'xtok', pw="114E284C-3C33-404C-AAC5-8B23B617E3F7")
xServerUrl = "https://xserver2-europe-eu-test.cloud.ptvgroup.com"
coord1 = get_geocoord("Pariser Platz 5, Berlin", credentials, xServerUrl)
coord2 = get_geocoord("Schloßbezirk 3, 76131 Karlsruhe", credentials, xServerUrl)
coord1, coord2

({'x': 13.379170455, 'y': 52.51694874}, {'x': 8.4014502032, 'y': 49.012371069})

In [3]:
# task 2
route = calc_route(coord1, coord2, xServerUrl)

In [4]:
# task 3

# install via 'pip3 install folium'
# examples: https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/
import folium

tiles = xServerUrl + '/services/rest/XMap/tile/{z}/{x}/{y}?storedProfile=silkysand';
m = folium.Map(location=order_coords(coord1), tiles=tiles, attr='PTV AG', zoom_start=6)
folium.Marker(order_coords(coord1), popup='<i>Berlin, Brandenburger Tor</i>').add_to(m)
folium.Marker(order_coords(coord2), popup='<b>Bundesverfassungsgericht</b>').add_to(m)
folium.PolyLine(order_polyline(route)).add_to(m)

m